In [10]:
import time
import pyodbc
import sqlalchemy
import pandas as pd
import queue 
from datetime import datetime
import numpy as np
from pandas import DataFrame
import pathlib
path = pathlib.Path().absolute()
# DB connection
database = 'KeeperDev'
username = 'sa'
password = 'Admin@12345'
server = 'projects.nimble.expert'
# Set up db connection
connection = pyodbc.connect("Driver={SQL Server};"
                                "Server=projects.nimble.expert;"
                                "Database=KeeperDev;"
                                "uid=sa;pwd=Admin@12345;IntegratedSecurity = false;")
cursor = connection.cursor()

# Create SQLAlchemy engine
engine = sqlalchemy.create_engine("mssql+pyodbc://" + username + ":" + password + "@" + server + "/" + database + "?driver=SQL+Server")

# Test SQL connection
engine.connect()

In [48]:
from IPython.display import HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Add a single batch

In [11]:
# Add single test batch
def addBatch(start, finish, default, w1, w2, w3, w4,copromo, colisting, cono):
    #df = pd.read_excel("aibot 15.09.2020.xlsx")
    df = pd.read_excel("demo_yara.xlsx")
    cursor.execute('SELECT MAX(ID) FROM dbo.Batch')
    (batch,) = cursor.fetchone()
    if batch == None:
        batch = 1
    else:
        batch += 1
    # Create values for dbo.Batch
    name = ("Test #" + str(batch))
    now = datetime.now()
    df['ID'] = batch
    batchID = df['ID']
    df.drop(labels=['ID'], axis=1,inplace = True)
    # Create Seqno.
    cursor.execute('SELECT MAX(Seqno) FROM dbo.Orders')
    (seq,) = cursor.fetchone()
    if seq == None:
        seq = 0
    df_i = df.index
    length = len(df_i)
    seqno = pd.Series(range(seq + 1 ,seq + length + 1))
    df.insert(0, 'batchId', batchID)
    #df.insert(1, 'Seqno', seqno)
    # Rename
    cursor.execute('''SET IDENTITY_INSERT dbo.Batch OFF;
    SET IDENTITY_INSERT dbo.Orders OFF;''')
    connection.commit()
    df = df.rename(columns={"covers demand till": "covers_demand_till"})
    # Insert into dbo.Batch
    cursor.execute('''
    SET IDENTITY_INSERT dbo.Batch  ON;

    INSERT INTO dbo.Batch(ID, Name, Excel, Sdate, Status, StartDate, FinishDate, Weight_bool, Weight_1, Weight_2, Weight_3, Weight_4, Cutoff_promo, Cutoff_listing, Cutoff_nodemand)
    VALUES (?,?,?,?, NULL,?,?,?,?,?,?,?,?,?,?);
    ''', batch, name,"Test.xlsx", now, start, finish, default , w1, w2, w3, w4,copromo, colisting, cono)
    connection.commit()
    # Insert into dbo.Orders
    cursor.execute('SET IDENTITY_INSERT dbo.Orders  ON;')
    df.to_sql("Orders", con=engine, if_exists='append', index=False)


In [19]:
# Add single batch 
start = "22-09-2020"
finish = "06-10-2020"
w1 = 0
w2 = 0
w3 = 0
w4 = 0
copromo = 90
colisting = 90
cono = 90
default = 1
addBatch(start, finish, default, w1, w2, w3, w4,copromo, colisting, cono)

### Display plan of last batch

In [53]:
# Show results from specific batch
batch = 1
cursor.execute('SELECT Status FROM dbo.Batch WHERE ID = ?',batch)
(status,) = cursor.fetchone()
if status != 'Finished':
    print("BatchID: " + str(batch), end = '')
    if status == 'Error':
        print(" has encountered an error, please check the error log")
    elif status == 'Processing':
        print(" is still in process")
    elif status == None:
        print(" is still queued")
elif status == "Finished":
    plan = pd.read_sql_query('SELECT * FROM dbo.Plans WHERE BatchID = (?)', engine, params=(batch,))
    display(plan)
    cursor.execute('SELECT Name FROM dbo.Batch WHERE ID = ?',batch)
    (name,) = cursor.fetchone()
    plan.to_excel(str(path) + "/Plans/" + name + ".xlsx")

,Work ctr,Order,Material,Description,Mould,Target qty,Originating document,Impact of color,Start Date,Start Time,...,Finish Time,Duration,Insert 1 (mould),Insert 2 (mould),Insert 3 (mould),Insert 4 (mould),Insert 5 (mould),StartPoint,BatchID,seqno
0,333,1255697,1011220200000,"Miska; 1,5L; Ø20X9CM",W011203-10,5520.0,,none,22-09-2020,01:55:00,...,09:17:00,01 07:22:00,,None,None,None,None,None,1,1
1,333,1238234,1011245200000,"Miska; 1,5L; Ø20X9CM",W011203-10,2500.0,,none,23-09-2020,11:17:00,...,01:29:00,14:12:00,,None,None,None,None,None,1,2
2,333,1257813,1011245200000,"Miska; 1,5L; Ø20X9CM",W011203-10,5520.0,,none,24-09-2020,01:29:00,...,08:51:00,01 07:22:00,,None,None,None,None,None,1,3
3,333,1256290,70669632000G520,"pokrywa fredo fresh pojemnik na żyw 0,1L",W366903-01,20160.0,,big,25-09-2020,12:01:00,...,15:18:00,02 03:17:00,I366901-01,None,None,None,None,None,1,4
4,333,1258924,70669632000G520,"pokrywa fredo fresh pojemnik na żyw 0,1L",W366903-01,30240.0,,none,27-09-2020,15:18:00,...,13:40:00,02 22:22:00,I366901-01,None,None,None,None,None,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,453,1257849,1078513400000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,4320.0,,none,02-10-2020,03:10:30,...,01:53:55,22:43:25,,None,None,None,None,None,1,340
340,453,1255877,1078566600000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,2700.0,,none,03-10-2020,01:53:55,...,16:00:26,14:06:31,,None,None,None,None,None,1,341
341,453,1258028,1078513400000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,180.0,4.00061e+007,none,03-10-2020,16:00:26,...,16:56:32,00:56:06,,None,None,None,None,None,1,342
342,453,1258828,1078513400000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,7560.0,,none,03-10-2020,16:56:32,...,10:33:44,01 17:37:00,,None,None,None,None,None,1,343


In [61]:
# Show results from latest batch
cursor.execute('SELECT MAX(ID) FROM dbo.Batch')
(batch,) = cursor.fetchone()
cursor.execute('SELECT Status FROM dbo.Batch WHERE ID = ?',batch)
(status,) = cursor.fetchone()
if status != 'Finished':
    print("BatchID: " + str(batch), end = '')
    if status == 'Error':
        print(" has encountered an error, please check the error log")
    elif status == 'Processing':
        print(" is still in process")
    elif status == None:
        print(" is still queued")
elif status == "Finished":
    plan = pd.read_sql_query('SELECT * FROM dbo.Plans WHERE BatchID = (?)', engine, params=(batch,))
    display(plan)
    cursor.execute('SELECT Name FROM dbo.Batch WHERE ID = ?',batch)
    (name,) = cursor.fetchone()
    plan.to_excel(str(path) + "/Plans/" + name + ".xlsx")

,Work ctr,Order,Material,Description,Mould,Target qty,Originating document,Impact of color,Start Date,Start Time,...,Finish Time,Duration,Insert 1 (mould),Insert 2 (mould),Insert 3 (mould),Insert 4 (mould),Insert 5 (mould),StartPoint,BatchID,seqno
0,333,1255697,1011220200000,"Miska; 1,5L; Ø20X9CM",W011203-10,5520.0,,none,22-09-2020,01:55:00,...,09:17:00,01 07:22:00,,None,None,None,None,None,6,345
1,333,1238234,1011245200000,"Miska; 1,5L; Ø20X9CM",W011203-10,2500.0,,none,23-09-2020,11:17:00,...,01:29:00,14:12:00,,None,None,None,None,None,6,346
2,333,1257813,1011245200000,"Miska; 1,5L; Ø20X9CM",W011203-10,5520.0,,none,24-09-2020,01:29:00,...,08:51:00,01 07:22:00,,None,None,None,None,None,6,347
3,333,1256290,70669632000G520,"pokrywa fredo fresh pojemnik na żyw 0,1L",W366903-01,20160.0,,big,25-09-2020,12:01:00,...,15:18:00,02 03:17:00,I366901-01,None,None,None,None,None,6,348
4,333,1258924,70669632000G520,"pokrywa fredo fresh pojemnik na żyw 0,1L",W366903-01,30240.0,,none,27-09-2020,15:18:00,...,13:40:00,02 22:22:00,I366901-01,None,None,None,None,None,6,349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,453,1257849,1078513400000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,4320.0,,none,02-10-2020,03:10:30,...,01:53:55,22:43:25,,None,None,None,None,None,6,684
340,453,1255877,1078566600000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,2700.0,,none,03-10-2020,01:53:55,...,16:00:26,14:06:31,,None,None,None,None,None,6,685
341,453,1258028,1078513400000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,180.0,4.00061e+007,none,03-10-2020,16:00:26,...,16:56:32,00:56:06,,None,None,None,None,None,6,686
342,453,1258828,1078513400000,aenna wanna na pranie; 32L; 55X40X23CM,W078503-10,7560.0,,none,03-10-2020,16:56:32,...,10:33:44,01 17:37:00,,None,None,None,None,None,6,687


### Batch Overview

In [78]:
# Show overview of batches
batches = pd.read_sql_query('SELECT * FROM dbo.Batch', engine)
display(batches)

,ID,Name,Excel,Sdate,Status,StartDate,FinishDate,Error code,Weight_bool,Weight_1,Weight_2,Weight_3,Weight_4,Cutoff_promo,Cutoff_listing,Cutoff_nodemand,Utilization,MaxChangeovers


### Remove test records

In [25]:
# Remove single batch
batchID = 6
cursor.execute('DELETE FROM dbo.Plans WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.Orders WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.Unknown WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.Stats WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.P_fill WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.P_color WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.P_filltotal WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.P_conflict WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.P_planned WHERE BatchID = ?;', batchID)
cursor.execute('DELETE FROM dbo.Batch WHERE ID = ?;', batchID)
connection.commit()


In [77]:
# Remove test records
cursor.execute('TRUNCATE TABLE dbo.Plans;')
cursor.execute('TRUNCATE TABLE dbo.Orders;')
cursor.execute('TRUNCATE TABLE dbo.Unknown;')
cursor.execute('TRUNCATE TABLE dbo.Stats;')
cursor.execute('TRUNCATE TABLE dbo.P_fill;')
cursor.execute('TRUNCATE TABLE dbo.P_color;')
cursor.execute('TRUNCATE TABLE dbo.P_filltotal;')
cursor.execute('TRUNCATE TABLE dbo.P_conflict;')
cursor.execute('TRUNCATE TABLE dbo.P_planned;')
cursor.execute('''
                DELETE FROM dbo.Batch
DBCC CHECKIDENT ('dbo.Batch',RESEED, 0)''')
connection.commit()
